In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Hacker_check').getOrCreate()

In [3]:
df = spark.read.csv('/home/ubuntu/Course_Notes/Spark_for_Machine_Learning/Clustering/hack_data.csv',inferSchema=True, header = True)

In [4]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
df.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [6]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [7]:
indexer = StringIndexer(inputCol= 'Location',outputCol='Loc_index')

In [10]:
indexed_df = indexer.fit(df).transform(df)

In [11]:
indexed_df.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|Loc_index|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|     54.0|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|     37.0|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|     63.0|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|    161.0|
|     

In [12]:
indexed_df.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'Loc_index']

In [13]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed',
 'Loc_index'],outputCol = 'Features')

In [18]:
indexed_df = assembler.transform(indexed_df)

In [14]:
from pyspark.ml.feature import StandardScaler

In [24]:
scaler = StandardScaler(inputCol= 'Features', outputCol="scaled_features")

In [25]:
scaled_features = scaler.fit(indexed_df).transform(indexed_df).select('scaled_features')

In [28]:
scaled_features.head(1)

[Row(scaled_features=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963, 1.059]))]

In [29]:
from pyspark.ml.clustering import KMeans

In [30]:
kmeans = KMeans(k=3,featuresCol= 'scaled_features')

In [31]:
model = kmeans.fit(scaled_features)

In [34]:
wsse = model.computeCost(scaled_features)
wsse

766.2083651605144

In [35]:
kmeans2 = KMeans(k=2, featuresCol= 'scaled_features')
model2 = kmeans2.fit(scaled_features)
wsse2 = model2.computeCost(scaled_features)
wsse2

934.2676092002957

In [51]:
model.clusterCenters()

[array([ 2.99991988,  2.92319035,  1.05261534,  3.20390443,  4.51321315,
         3.28474   ,  1.23589466]),
 array([ 1.21780112,  1.37901802,  1.99757683,  1.37198977,  2.55237797,
         5.29152222,  1.36502738]),
 array([ 1.30217042,  1.25830099,  0.        ,  1.35793211,  2.57251009,
         5.24230473,  1.2626253 ])]

In [57]:
model.transform(scaled_features).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [60]:
model2.transform(scaled_features).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



There are only 2 hackers here